## Setup

In [36]:
import pandas as pd
from plotly import graph_objects as go
# import matplotlib.pyplot as plt
# import seaborn as sns
from scipy import stats as st
from statsmodels.stats.proportion import proportions_ztest
import datetime
import numpy as np
# import display and markdown
from IPython.display import display, Markdown

%cd "t:\My Drive\wdir\s14-online_store"
%cd "drive/MyDrive/wdir/s14-online_store/"

%pwd

t:\My Drive\wdir\s14-online_store
[WinError 3] The system cannot find the path specified: 'drive/MyDrive/wdir/s14-online_store/'
t:\My Drive\wdir\s14-online_store


't:\\My Drive\\wdir\\s14-online_store'

## Load Files

In [32]:
%%bash
ls -l datasets/raw/

ab_project_marketing_events_us.csv
final_ab_events_upd_us.csv
final_ab_new_users_upd_us.csv
final_ab_participants_upd_us.csv
final_ab_events_upd_us.csv
final_ab_new_users_upd_us.csv
final_ab_participants_upd_us.csv


In [40]:
# Cargar los archivos de datos
mark_events = pd.read_csv('datasets/raw/ab_project_marketing_events_us.csv', parse_dates=['start_dt', 'finish_dt'])
display(Markdown("### Mark events"))
display(mark_events.head(), mark_events.info())
users = pd.read_csv('datasets/raw/final_ab_new_users_upd_us.csv', parse_dates=['first_date'])
display(Markdown("### Users"))
display(users.head(), users.info())

ab_events = pd.read_csv('datasets/raw/final_ab_events_upd_us.csv', parse_dates=['event_dt'])
display(Markdown("### AB events"))
display(ab_events.head(), ab_events.info())

participants = pd.read_csv('datasets/raw/final_ab_participants_upd_us.csv')
display(Markdown("### Participants"))
display(participants.head(), participants.info())


assert users.user_id.value_counts().max() == 1, "Each user can only have one record"

### Mark events

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


None

### Users

...

None

### Participants

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test


None

In [33]:
# downcast data types of numerical columns
def downcast_dtypes(df):
    fcols = df.select_dtypes('float').columns
    icols = df.select_dtypes('integer').columns

    df[fcols] = df[fcols].apply(pd.to_numeric, downcast='float')
    df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')

for df in [mark_events, users, ab_events, participants]:
    downcast_dtypes(df)

In [23]:
# Verificar usuarios en ambos grupos
overlap_users = participants[participants.duplicated('user_id', keep=False)]
print(f"Usuarios en ambos grupos: {overlap_users.shape[0]}")

# drop overlaping users
participants = participants.drop(overlap_users.index)
assert participants.user_id.value_counts().max() == 1, "Each user can only have one record"

Usuarios en ambos grupos: 1774


Aplicamos los filtros necesarios

In [24]:
# Filtro por región "UE"
mark_events_ue = mark_events[mark_events['regions'].str.contains("UE")]

# Verificar fechas de aplicación
start_date = datetime.datetime.strptime("2020-12-07", "%Y-%m-%d")
end_date = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")
ab_events = ab_events[(ab_events['event_dt'] >= start_date) & (ab_events['event_dt'] <= end_date)]

# Filtro de participantes en el test
participants = participants[participants['ab_test'] == 'recommender_system_test']
ab_events = ab_events[ab_events['user_id'].isin(participants['user_id'])]

# Filtrar eventos relevantes
relevant_events = ['product_page', 'product_cart', 'purchase']
ab_events = ab_events[ab_events['event_name'].isin(relevant_events)]

# Verificar primeros 14 días desde el registro
ab_events = ab_events.merge(users[['user_id', 'first_date']], on='user_id')
ab_events['days_since_registration'] = (ab_events['event_dt'] - ab_events['first_date']).dt.days
ab_events = ab_events[ab_events['days_since_registration'] <= 14]

# Eliminando la columna auxiliar 'days_since_registration' para limpieza de datos
ab_events.drop(columns=['days_since_registration'], inplace=True)

In [ ]:
%%bash
# Create the folder clean
mkdir --parents "datasets/clean"
# Create the folder results
mkdir --parents "datasets/results"

In [25]:
# Save the dataframes as parquet files
mark_events_ue.to_parquet('datasets/clean/mark_events_ue.parquet')
users.to_parquet('datasets/clean/users.parquet')
ab_events.to_parquet('datasets/clean/ab_events.parquet')
participants.to_parquet('datasets/clean/participants.parquet')

A subdirectory or file r.\datasets\clean already exists.
